# Mapping Challenge
**Treinamento de rede neural para localização de casas em imagens Satelitais**

Como proposto na Prova técnica do FIESC-Senai, será treinado uma rede neural Fast R-CNN

# Inicialização do Google Drive
**Extração do Dataset do Zip**

In [ ]:
# Montar o Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Caminho para o arquivo ZIP no Google Drive
zip_path = '/content/drive/My Drive/building.zip'

# Diretório de destino para extrair os arquivos
extract_path = '/content/dataset'

# Extrair o arquivo ZIP
import zipfile
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Verificar os arquivos extraídos
import os
os.listdir(extract_path)

Mounted at /content/drive


['annotation.json', 'annotation-small.json', 'train']

# Pré-Processamento do Dataset

In [ ]:
import json

# Carregar o arquivo annotations.json
annotations_file = '/content/drive/My Drive/annotation.json'
with open(annotations_file, 'r') as f:
    data = json.load(f)

# Iterar sobre as informações das imagens
for img_info in data['images']:
    img_id = img_info['id']
    width = img_info['width']
    height = img_info['height']

    # Verificar se a largura e altura são diferentes de 300
    if width != 300 or height != 300:
        img_path = os.path.join(data_dir, img_info['file_name'])
        print(f"Image ID: {img_id}, Width: {width}, Height: {height}, Path: {img_path}")


In [ ]:


# Carregar o arquivo de anotações
annotations_file = '/content/drive/My Drive/annotation.json'
with open(annotations_file, 'r') as f:
    annotations = json.load(f)

# Criar um conjunto de todos os IDs de imagem
image_ids = set(img_info['id'] for img_info in annotations['images'])

# Criar um conjunto de todos os IDs de imagem anotados
annotated_image_ids = set(img_info['image_id'] for img_info in annotations['annotations'])

# Encontrar os IDs de imagem sem anotações correspondentes
missing_annotations = image_ids - annotated_image_ids

# Printar os IDs de imagem sem anotações correspondentes
if missing_annotations:
    print("IDs de imagem sem anotações correspondentes:")
    for img_id in missing_annotations:
        print(img_id)
else:
    print("Todas as imagens têm anotações correspondentes.")

Todas as imagens têm anotações correspondentes.


In [ ]:

# Caminho para o arquivo de anotação JSON original
annotation_file = '/content/drive/My Drive/annotation.json'

# Caminho para o novo arquivo de anotação JSON sem as anotações inválidas
new_annotation_file = '/content/drive/My Drive/n-annotation.json'

# Carregar o arquivo de anotação JSON
with open(annotation_file, "r") as f:
    data = json.load(f)

# Inicializar lista para armazenar IDs de imagens com anotações válidas
valid_image_ids = []

# Inicializar dicionário para armazenar anotações válidas
valid_annotations = {}

# Percorrer todas as imagens no conjunto de dados
for img_id, img_info in data["images"].items():
    # Obter IDs das anotações para a imagem atual
    ann_ids = data["annotations"].get(img_id, [])
    # Verificar cada anotação para a imagem atual
    for ann_id in ann_ids:
        # Verificar se a anotação tem o campo 'bbox'
        if 'bbox' not in data["annotations"][ann_id]:
            # Se não tiver, continuar para a próxima anotação
            continue
        # Obter largura e altura da caixa delimitadora
        x, y, w, h = data["annotations"][ann_id]['bbox']
        # Verificar se a largura e a altura são positivas
        if w > 0 and h > 0:
            # Se forem, adicionar ID da imagem à lista de IDs de imagens com anotações válidas
            valid_image_ids.append(img_id)
            # Adicionar a anotação ao dicionário de anotações válidas
            valid_annotations[ann_id] = data["annotations"][ann_id]

# Atualizar o campo images no novo conjunto de dados com as imagens válidas
data["images"] = {img_id: img_info for img_id, img_info in data["images"].items() if img_id in valid_image_ids}
# Atualizar o campo annotations no novo conjunto de dados com as anotações válidas
data["annotations"] = valid_annotations

# Salvar as anotações atualizadas em um novo arquivo JSON
with open(new_annotation_file, "w") as f:
    json.dump(data, f)

print("Anotações válidas foram salvas em", new_annotation_file)

# Inicialização do Dataset

In [ ]:
import os
import torch
import torch.utils.data
import torchvision
from PIL import Image
from pycocotools.coco import COCO

class myOwnDataset(torch.utils.data.Dataset):
    def __init__(self, root, annotation, transforms=None):
        self.root = root
        self.transforms = transforms
        self.coco = COCO(annotation)
        self.ids = list(sorted(self.coco.imgs.keys()))

    def __getitem__(self, index):
        # Objeto da Biblioteca Coco
        coco = self.coco
        # Index das Imagens
        img_id = self.ids[index]
        #  Index das Anotações
        ann_ids = coco.getAnnIds(imgIds=img_id)
        coco_annotation = coco.loadAnns(ann_ids)
        path = coco.loadImgs(img_id)[0]['file_name']
        img = Image.open(os.path.join(self.root, path))
        # Número de Objetos anotados na imagem
        num_objs = len(coco_annotation)

        # Bounding boxes das imagens
        # Formato biblioteca coco bbox = [xmin, ymin, width, height]
        # Convertendo para o formato pytorch = [xmin, ymin, xmax, ymax]
        boxes = []
        for i in range(num_objs):
            xmin = coco_annotation[i]['bbox'][0]
            ymin = coco_annotation[i]['bbox'][1]
            xmax = xmin + coco_annotation[i]['bbox'][2]
            ymax = ymin + coco_annotation[i]['bbox'][3]
            boxes.append([xmin, ymin, xmax, ymax])
        boxes = torch.as_tensor(boxes, dtype=torch.float32)


        labels = torch.ones((num_objs,), dtype=torch.int64)

        img_id = torch.tensor([img_id])
        # Size of bbox (Rectangular)
        areas = []
        for i in range(num_objs):
            areas.append(coco_annotation[i]['area'])
        areas = torch.as_tensor(areas, dtype=torch.float32)

        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        # Convertendo anotação para formato de dicionário
        my_annotation = {}
        my_annotation["boxes"] = boxes
        my_annotation["labels"] = labels
        my_annotation["image_id"] = img_id
        my_annotation["area"] = areas
        my_annotation["iscrowd"] = iscrowd

        if self.transforms is not None:
            img = self.transforms(img)

        return img, my_annotation

    def __len__(self):
        return len(self.ids)

In [ ]:
def get_transform():
    custom_transforms = []
    custom_transforms.append(torchvision.transforms.ToTensor())
    return torchvision.transforms.Compose(custom_transforms)

# Inicialização do DataLoader

In [ ]:

train_data_dir = r"/content/dataset/train/images"
train_coco = r"/content/drive/MyDrive/n-annotation.json"


my_dataset = myOwnDataset(root=train_data_dir,
                          annotation=train_coco,
                          transforms=get_transform()
                          )

#Adaptação do Dataset para rodar um conjunto menor de imagens para teste do código
image_indices = list(range(100))
my_dataset2 = data.Subset(my_dataset, image_indices)

# collate_fn  custom  para o formato COCO
def collate_fn(batch):
    return tuple(zip(*batch))

train_batch_size = 8


data_loader = torch.utils.data.DataLoader(my_dataset2,
                                          batch_size=train_batch_size,
                                          shuffle=True,
                                          num_workers=2,
                                          collate_fn=collate_fn)

loading annotations into memory...
Done (t=34.36s)
creating index...
index created!


# Treinamento com Early Stopping

In [ ]:
import torch
import torch.utils.data
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# Incialização do modelo Fast RCNN treinado do MS COCO
def get_model_instance_segmentation(num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

num_classes = 2
num_epochs = 10
patience = 10
best_loss = float('inf')
best_model_state = None


model = get_model_instance_segmentation(num_classes)


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)


params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for imgs, annotations in data_loader:
        imgs = list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
        loss_dict = model(imgs, annotations)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        running_loss += losses.item()

    epoch_loss = running_loss / len(data_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss}')

    if epoch_loss < best_loss:
        best_loss = epoch_loss

        best_model_state = model.state_dict()

        patience_counter = 0
    else:

        patience_counter += 1

    if patience_counter >= patience:
        print(f'Early stopping após {patience} épocas sem melhorias.')
        break

torch.save(best_model_state, 'best_model.pt')